In this note bookbook, we will create and save images by reading parquet files

In [1]:
%matplotlib inline 
# If we don't do this then image will open as pop-up and not in notebook

In [2]:
#!pip3 install tensorflow

In [2]:
!pip3 show tensorflow

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/ubuntu/.local/lib/python3.6/site-packages
Requires: termcolor, grpcio, keras-preprocessing, wheel, absl-py, tensorboard, google-pasta, six, astor, wrapt, gast, keras-applications, protobuf, tensorflow-estimator, numpy


In [4]:
# !export PYTHONPATH=/home/ubuntu/.local/lib/python3.6/site-packages:$PYTHONPATH.
# ^ This makes all other libraries inaccessible

In [3]:
import tensorflow as tf

In [94]:
tf.enable_eager_execution()

In [4]:
#import pyarrow.parquet as pq 
import pandas as pd
import numpy as np
from PIL import Image as im
import matplotlib as plt
from matplotlib.pyplot import imshow

In [5]:
import wget
import time
import os
import copy
from skimage import io

In [7]:
!ls -l --block=M /home/ubuntu/datasets/mammography/

total 2951M
drwxr-xr-x 2 ubuntu ubuntu    1M Jan 28 21:05 cv10_data
-rw-r--r-- 1 ubuntu ubuntu    1M Sep 30 23:37 cv10_labels.npy
-rw-rw-r-- 1 ubuntu ubuntu 2951M Jan 28 20:52 ddsm-mammography.zip
drwxr-xr-x 2 ubuntu ubuntu    1M Jan 28 21:05 test10_data
-rw-r--r-- 1 ubuntu ubuntu    1M Sep 30 23:38 test10_labels.npy
drwxr-xr-x 2 ubuntu ubuntu    1M Jan 28 21:05 training10_0
drwxr-xr-x 2 ubuntu ubuntu    1M Jan 28 21:05 training10_1
drwxr-xr-x 2 ubuntu ubuntu    1M Jan 28 21:05 training10_2
drwxr-xr-x 2 ubuntu ubuntu    1M Jan 28 21:05 training10_3
drwxr-xr-x 2 ubuntu ubuntu    1M Jan 28 21:05 training10_4


As tensorflow could not be loaded we are going to use tfrecord library:
https://github.com/vahidk/tfrecord

### From Kaggle kernel

In [90]:
# make sure everything was written properly by reading it back out
def read_and_decode_single_example(filenames):
#     filename_queue = tf.train.string_input_producer(filenames, num_epochs=1)
#     reader = tf.TFRecordReader()    
#     _, serialized_example = reader.read(filename_queue)
    dataset = tf.data.TFRecordDataset(filenames)
    raw_example = next(iter(dataset))
    parsed = tf.train.Example.FromString(raw_example.numpy())
    
    features = tf.parse_single_example(
        parsed,
        features={
            'label_normal': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        })
    
    # now return the converted data
    label = features['label_normal']
    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.reshape(image, [299, 299, 1])
    
    return label, image

In [96]:
label, image = read_and_decode_single_example(["/home/ubuntu/datasets/mammography/training10_0/training10_0.tfrecords"])
images_batch, labels_batch = tf.train.batch([image, label], batch_size=16, capacity=2000)
global_step = tf.Variable(0, trainable=False)

AttributeError: 'Tensor' object has no attribute 'numpy'

In [89]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for j in range(3):
        la_b, im_b = sess.run([labels_batch, images_batch])
        
        for i in range(8):
            plt.imshow(im_b[i].reshape([299,299]))
            plt.title("Label: " + str(la_b[i]))
            plt.show()
            
    coord.request_stop()
    
    # Wait for threads to stop
    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.NotFoundError'>, ../input/training10_1/training10_1.tfrecords; No such file or directory
	 [[{{node ReaderReadV2}}]]


OutOfRangeError: FIFOQueue '_7_batch_1/fifo_queue' is closed and has insufficient elements (requested 15, current size 0)
	 [[node batch_1 (defined at <ipython-input-88-9b3354eeaab1>:2) ]]

Original stack trace for 'batch_1':
  File "/home/ubuntu/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/

In [28]:
def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        features={
            'label_normal': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        })
    # now return the converted data
    label = features['label_normal']
    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.reshape(image, [299, 299, 1])

In [7]:
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)

In [10]:
dataset = tf.data.TFRecordDataset(filenames="/home/ubuntu/datasets/mammography/training10_0/training10_0.tfrecords")

In [8]:
dataset

<TFRecordDatasetV1 shapes: (), types: tf.string>

In [11]:
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())

In [12]:
parsed

features {
  feature {
    key: "filename"
    value {
      bytes_list {
        value: "N\000\000\000O\000\000\000R\000\000\000M\000\000\000A\000\000\000L\000\000\000"
      }
    }
  }
  feature {
    key: "image"
    value {
      bytes_list {
        value: "BDJOTVUUXZ\\]\\^_`_^XYYYVXYY\\YYZWVZWWWWYY]\\]cfjlkikiijknnkloponooorrrsrrqohfgihgeciljkjjkkkkihhhgghhjlopqrruwxrrspolkklpoprrpqqqonqqquwx|}|}}\200\200}}~\200}\201\204\201\200~\200\201\201\200\200}{z}\200\200\203\204\202\200\202\205\213\214\214\217\223\223\225\223\223\223\225\227\227\230\227\225\226\225\225\227\230\227\230\227\223\223\224\223\225\226\223\220\222\224\223\224\225\226\224\223\223\220\217\214\213\207\211\207\202\202\203\204\202\205\206\213\214\214\212\211\211\207\206\207\205\206\212\214\214\215\211\206\206\211\213\207\207\207\212\205\201\201~\202\204\206\206\205\206\212\212\213\214\211\207\207\212\212\207\203\205\212\213\212\213\215\213\206\206\205\203\206\211\205\204\203\204\205\207\204\206\212\213\216\213\211\21

In [26]:
type(parsed)

tensorflow.core.example.example_pb2.Example

In [64]:
i = 1

In [65]:
for example in tf.python_io.tf_record_iterator("/home/ubuntu/datasets/mammography/training10_0/training10_0.tfrecords"):
    #print(tf.train.Example.FromString(example))
    jsonMessage = MessageToJson(tf.train.Example.FromString(example))
    i += 1
    if i == 3:
        break

In [66]:
jsonMessage

'{\n  "features": {\n    "feature": {\n      "label": {\n        "int64List": {\n          "value": [\n            "0"\n          ]\n        }\n      },\n      "image": {\n        "bytesList": {\n          "value": [\n            "..."\n          ]\n        }\n      },\n      "label_normal": {\n        "int64List": {\n          "value": [\n            "0"\n          ]\n        }\n      },\n      "filename": {\n        "bytesList": {\n          "value": [\n            "TgAAAE8AAABSAAAATQAAAEEAAABMAAAA"\n          ]\n        }\n      }\n    }\n  }\n}'

In [70]:
jsonMessage[5]

'f'

In [37]:
from google.protobuf.json_format import MessageToJson

In [38]:
jsonMessage = MessageToJson(tf.train.Example.FromString(parsed))

TypeError: a bytes-like object is required, not 'Example'

In [50]:
for example in tf.python_io.tf_record_iterator("/home/ubuntu/datasets/mammography/training10_0/training10_0.tfrecords"):
    result = tf.train.SequenceExample.FromString(example)
    break
print(result)

context {
  feature {
    key: "filename"
    value {
      bytes_list {
        value: "N\000\000\000O\000\000\000R\000\000\000M\000\000\000A\000\000\000L\000\000\000"
      }
    }
  }
  feature {
    key: "image"
    value {
      bytes_list {
        value: "BDJOTVUUXZ\\]\\^_`_^XYYYVXYY\\YYZWVZWWWWYY]\\]cfjlkikiijknnkloponooorrrsrrqohfgihgeciljkjjkkkkihhhgghhjlopqrruwxrrspolkklpoprrpqqqonqqquwx|}|}}\200\200}}~\200}\201\204\201\200~\200\201\201\200\200}{z}\200\200\203\204\202\200\202\205\213\214\214\217\223\223\225\223\223\223\225\227\227\230\227\225\226\225\225\227\230\227\230\227\223\223\224\223\225\226\223\220\222\224\223\224\225\226\224\223\223\220\217\214\213\207\211\207\202\202\203\204\202\205\206\213\214\214\212\211\211\207\206\207\205\206\212\214\214\215\211\206\206\211\213\207\207\207\212\205\201\201~\202\204\206\206\205\206\212\212\213\214\211\207\207\212\212\207\203\205\212\213\212\213\215\213\206\206\205\203\206\211\205\204\203\204\205\207\204\206\212\213\216\213\211\211

In [41]:
result.context.feature["filename"], result.context.feature["label"]

(bytes_list {
   value: "N\000\000\000O\000\000\000R\000\000\000M\000\000\000A\000\000\000L\000\000\000"
 }, int64_list {
   value: 0
 })

In [56]:
tt= result.context.feature["filename"]
tt

bytes_list {
  value: "N\000\000\000O\000\000\000R\000\000\000M\000\000\000A\000\000\000L\000\000\000"
}

### DataBrick tutorial

In [71]:
dataset = tf.data.TFRecordDataset("/home/ubuntu/datasets/mammography/training10_0/training10_0.tfrecords")

In [73]:
def decode(serialized_example):
    """
    Parses an image and label from the given `serialized_example`.
    It is used as a map function for `dataset.map`
    """
    IMAGE_SIZE = 299
    IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE
  
    # 1. define a parser
    features = tf.parse_single_example(
        serialized_example,
        # Defaults are not specified since both keys are required.
        features={
            'image': tf.FixedLenFeature([], tf.string),
            'label_normal': tf.FixedLenFeature([], tf.int64),
        })

    # 2. Convert the data
    image = tf.decode_raw(features['image'], tf.uint8)
    label = tf.cast(features['label_normal'], tf.int32)
    # 3. reshape
    image.set_shape((IMAGE_PIXELS))
    return image, label

In [74]:
dataset = dataset.map(decode)

In [75]:
dataset

<DatasetV1Adapter shapes: ((89401,), ()), types: (tf.uint8, tf.int32)>

In [76]:
def normalize(image, label):
    """Convert `image` from [0, 255] -> [-0.5, 0.5] floats."""
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    return image, label

In [77]:
dataset = dataset.map(normalize)

In [79]:
dataset = dataset.batch(100)

In [80]:
iterator = dataset.make_one_shot_iterator()
image_batch, label_batch = iterator.get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [82]:
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)

In [83]:
image_batch, label_batch = session.run([image_batch, label_batch])
print(image_batch.shape)
print(label_batch.shape)

RuntimeError: The Session graph is empty.  Add operations to the graph before calling run().

In [46]:
bytelist = result.context.feature["label"]
print(''.join(bytelist).decode('utf-8'))

TypeError: can only join an iterable

In [30]:
# Create a description of the features.
feature_description = {
    'label_normal': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'image': tf.io.FixedLenFeature([], tf.string, default_value=''),
}

def _parse_function(example_proto):
  # Parse the input `tf.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)


In [31]:
parsed_dataset = dataset.map(_parse_function)
parsed_dataset

<DatasetV1Adapter shapes: {image: (), label_normal: ()}, types: {image: tf.string, label_normal: tf.int64}>

In [34]:
for parsed_record in parsed_dataset.take(2):
    print(repr(parsed_record))

{'image': <tf.Tensor: id=124, shape=(), dtype=string, numpy=b'BDJOTVUUXZ\\]\\^_`_^XYYYVXYY\\YYZWVZWWWWYY]\\]cfjlkikiijknnkloponooorrrsrrqohfgihgeciljkjjkkkkihhhgghhjlopqrruwxrrspolkklpoprrpqqqonqqquwx|}|}}\x80\x80}}~\x80}\x81\x84\x81\x80~\x80\x81\x81\x80\x80}{z}\x80\x80\x83\x84\x82\x80\x82\x85\x8b\x8c\x8c\x8f\x93\x93\x95\x93\x93\x93\x95\x97\x97\x98\x97\x95\x96\x95\x95\x97\x98\x97\x98\x97\x93\x93\x94\x93\x95\x96\x93\x90\x92\x94\x93\x94\x95\x96\x94\x93\x93\x90\x8f\x8c\x8b\x87\x89\x87\x82\x82\x83\x84\x82\x85\x86\x8b\x8c\x8c\x8a\x89\x89\x87\x86\x87\x85\x86\x8a\x8c\x8c\x8d\x89\x86\x86\x89\x8b\x87\x87\x87\x8a\x85\x81\x81~\x82\x84\x86\x86\x85\x86\x8a\x8a\x8b\x8c\x89\x87\x87\x8a\x8a\x87\x83\x85\x8a\x8b\x8a\x8b\x8d\x8b\x86\x86\x85\x83\x86\x89\x85\x84\x83\x84\x85\x87\x84\x86\x8a\x8b\x8e\x8b\x89\x89\x8b\x8a\x8b\x8b@AFNRTVXYX\\__`a``a^ZXXYYXY\\ZZYXWXWVVXZY^Z^achlnjlnjinonlnllooonopsurqrqnkihjigfeikkknllnjjiijifjljjnppoqprwwsrtqnljjlpppoqqpqqpoqrrtwz}|}~~\x80~{}\x80\x81~~\x83\x85\x83~}}|}~~{{}\x80\

In [35]:
tt = parsed_dataset.take(1)

In [36]:
example = tf.train.Example()
example.ParseFromString(tt.numpy())
print(example)

AttributeError: 'DatasetV1Adapter' object has no attribute 'numpy'

In [25]:
features={
            'label_normal': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        }

In [27]:
parsed_example = tf.parse_single_example(serialized=parsed, features=features)

ValueError: Attempt to convert a value (features {
  feature {
    key: "filename"
    value {
      bytes_list {
        value: "N\000\000\000O\000\000\000R\000\000\000M\000\000\000A\000\000\000L\000\000\000"
      }
    }
  }
  feature {
    key: "image"
    value {
      bytes_list {
        value: "BDJOTVUUXZ\\]\\^_`_^XYYYVXYY\\YYZWVZWWWWYY]\\]cfjlkikiijknnkloponooorrrsrrqohfgihgeciljkjjkkkkihhhgghhjlopqrruwxrrspolkklpoprrpqqqonqqquwx|}|}}\200\200}}~\200}\201\204\201\200~\200\201\201\200\200}{z}\200\200\203\204\202\200\202\205\213\214\214\217\223\223\225\223\223\223\225\227\227\230\227\225\226\225\225\227\230\227\230\227\223\223\224\223\225\226\223\220\222\224\223\224\225\226\224\223\223\220\217\214\213\207\211\207\202\202\203\204\202\205\206\213\214\214\212\211\211\207\206\207\205\206\212\214\214\215\211\206\206\211\213\207\207\207\212\205\201\201~\202\204\206\206\205\206\212\212\213\214\211\207\207\212\212\207\203\205\212\213\212\213\215\213\206\206\205\203\206\211\205\204\203\20

In [28]:
label = parsed['label_normal']
#image = tf.decode_raw(features['image'], tf.uint8)
#image = tf.reshape(image, [299, 299, 1])

TypeError: 'Example' object is not subscriptable

In [29]:
parsed.features

feature {
  key: "filename"
  value {
    bytes_list {
      value: "N\000\000\000O\000\000\000R\000\000\000M\000\000\000A\000\000\000L\000\000\000"
    }
  }
}
feature {
  key: "image"
  value {
    bytes_list {
      value: "BDJOTVUUXZ\\]\\^_`_^XYYYVXYY\\YYZWVZWWWWYY]\\]cfjlkikiijknnkloponooorrrsrrqohfgihgeciljkjjkkkkihhhgghhjlopqrruwxrrspolkklpoprrpqqqonqqquwx|}|}}\200\200}}~\200}\201\204\201\200~\200\201\201\200\200}{z}\200\200\203\204\202\200\202\205\213\214\214\217\223\223\225\223\223\223\225\227\227\230\227\225\226\225\225\227\230\227\230\227\223\223\224\223\225\226\223\220\222\224\223\224\225\226\224\223\223\220\217\214\213\207\211\207\202\202\203\204\202\205\206\213\214\214\212\211\211\207\206\207\205\206\212\214\214\215\211\206\206\211\213\207\207\207\212\205\201\201~\202\204\206\206\205\206\212\212\213\214\211\207\207\212\212\207\203\205\212\213\212\213\215\213\206\206\205\203\206\211\205\204\203\204\205\207\204\206\212\213\216\213\211\211\213\212\213\213@AFNRTVXYX\\__`a``a^

In [20]:
label = features['label_normal']
image = tf.decode_raw(features['image'], tf.uint8)
image = tf.reshape(image, [299, 299, 1])

TypeError: 'Example' object is not subscriptable

In [18]:
features = tf.parse_single_example(
        parsed,
        features={
            'label_normal': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        })
# now return the converted data
label = features['label_normal']
image = tf.decode_raw(features['image'], tf.uint8)
image = tf.reshape(image, [299, 299, 1])

ValueError: Attempt to convert a value (features {
  feature {
    key: "filename"
    value {
      bytes_list {
        value: "N\000\000\000O\000\000\000R\000\000\000M\000\000\000A\000\000\000L\000\000\000"
      }
    }
  }
  feature {
    key: "image"
    value {
      bytes_list {
        value: "BDJOTVUUXZ\\]\\^_`_^XYYYVXYY\\YYZWVZWWWWYY]\\]cfjlkikiijknnkloponooorrrsrrqohfgihgeciljkjjkkkkihhhgghhjlopqrruwxrrspolkklpoprrpqqqonqqquwx|}|}}\200\200}}~\200}\201\204\201\200~\200\201\201\200\200}{z}\200\200\203\204\202\200\202\205\213\214\214\217\223\223\225\223\223\223\225\227\227\230\227\225\226\225\225\227\230\227\230\227\223\223\224\223\225\226\223\220\222\224\223\224\225\226\224\223\223\220\217\214\213\207\211\207\202\202\203\204\202\205\206\213\214\214\212\211\211\207\206\207\205\206\212\214\214\215\211\206\206\211\213\207\207\207\212\205\201\201~\202\204\206\206\205\206\212\212\213\214\211\207\207\212\212\207\203\205\212\213\212\213\215\213\206\206\205\203\206\211\205\204\203\20

In [5]:
filename_queue = tf.train.string_input_producer(["~/datasets/mammography/training10_0/training10_0.tfrecords"])

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [20]:
filename_queue

In [22]:
reader = tf.TFRecordReader()

In [23]:
_, serialized_example = reader.read(filename_queue)

In [30]:
serialized_example

<tf.Tensor 'ReaderReadV2_4:1' shape=() dtype=string>

In [31]:
features = tf.parse_single_example(
        serialized_example,
        features={
            'label_normal': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        })

In [10]:
label = features['label_normal']
image = tf.decode_raw(features['image'], tf.uint8)
image = tf.reshape(image, [299, 299, 1])

TypeError: 'TextLineDatasetV1' object is not subscriptable

In [33]:
label

<tf.Tensor 'ParseSingleExample_6/ParseSingleExample:1' shape=() dtype=int64>

In [37]:
#tt = image.eval()

In [38]:
image, type(image), image.numpy()

AttributeError: 'Tensor' object has no attribute 'numpy'

In [ ]:
features = tf.parse_single_example(
        serialized_example,
        features={
            'label_normal': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        })
    
    # now return the converted data
    label = features['label_normal']
    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.reshape(image, [299, 299, 1])

In [21]:
image, label, height, width, depth = read_and_decode(filename_queue)

TypeError: cannot unpack non-iterable NoneType object

In [29]:
with tf.Session() as sess:
    filename_queue = tf.train.string_input_producer(["~/datasets/mammography/training10_0/training10_0.tfrecords"])
    image, label, height, width, depth = read_and_decode(filename_queue)
    image = tf.reshape(image, tf.stack([height, width, 3]))
    image.set_shape([299,299,3])
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(1000):
        example, l = sess.run([image, label])
        print (example,l)
    coord.request_stop()
    coord.join(threads)

TypeError: cannot unpack non-iterable NoneType object

In [9]:
# make sure everything was written properly by reading it back out
def read_and_decode_single_example(filenames):
    filename_queue = tf.train.string_input_producer(filenames, num_epochs=1)
    
    reader = tf.TFRecordReader()
    
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
        serialized_example,
        features={
            'label_normal': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        })
    
    # now return the converted data
    label = features['label_normal']
    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.reshape(image, [299, 299, 1])
    
    return label, image

In [10]:
label, image = read_and_decode_single_example(["~/datasets/mammography/training10_0/training10_0.tfrecords", "~/datasets/mammography/training10_1/training10_1.tfrecords"])

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Prefer Dataset.range instead.
Instructions for updating:
Prefer Dataset.range instead.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based inp

In [12]:
label.shape

TensorShape([])

In [13]:
images_batch, labels_batch = tf.train.batch([image, label], batch_size=16, capacity=2000)
global_step = tf.Variable(0, trainable=False)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).


In [11]:
labels_batch.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'batch:1' shape=(16,) dtype=int64>>

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for j in range(3):
        la_b, im_b = sess.run([labels_batch, images_batch])
        
        for i in range(8):
            plt.imshow(im_b[i].reshape([299,299]))
            plt.title("Label: " + str(la_b[i]))
            plt.show()
            
    coord.request_stop()
    
    # Wait for threads to stop
    coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.NotFoundError'>, ~/datasets/mammography/training10_1/training10_1.tfrecords; No such file or directory
	 [[{{node ReaderReadV2}}]]


OutOfRangeError: FIFOQueue '_1_batch/fifo_queue' is closed and has insufficient elements (requested 16, current size 0)
	 [[node batch (defined at <ipython-input-13-13a138149fd2>:1) ]]

Original stack trace for 'batch':
  File "/home/ubuntu/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/home/u

### The Train Files

In [6]:
def show_image_resized(dataframe, n):
    img_name = dataframe.iloc[n, 0]
    img_data = np.array(dataframe.iloc[n, 1:])
    img_data = img_data.astype('float').reshape(137,236)
    #img_data = im.resize img_data
    img_data = cv2.resize(img_data, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    # normalizing the image
    img_data = (img_data*(255.0/img_data.max())).astype(np.uint8)
    # show
    imshow(img_data, cmap='gray')

In [1]:
# show_image_resized(train, 69)

In [7]:
import cv2

In [8]:
img_dim = 224

### Reading and converting training data

In [ ]:
train_0 = pq.read_pandas('/home/ubuntu/datasets/bengali-ai/train_image_data_0.parquet').to_pandas()

In [ ]:
dataframe = train_0

for row in range(len(dataframe)):
    img_name = dataframe.iloc[row, 0]
    img_data = np.array(dataframe.iloc[row, 1:])
    img_data = img_data.astype('float').reshape(137,236)
    img_data = cv2.resize(img_data, dsize=(img_dim, img_dim), interpolation=cv2.INTER_CUBIC)
    # normalizing the image
    img_data = (img_data*(255.0/img_data.max())).astype(np.uint8)
    # save
    plt.image.imsave('/home/ubuntu/datasets/bengali-ai/training_images/train_0/'+img_name+'.png', img_data, cmap='gray')

In [94]:
del train_0

In [10]:
train_1 = pq.read_pandas('/home/ubuntu/datasets/bengali-ai/train_image_data_1.parquet').to_pandas()

In [11]:
dataframe = train_1

for row in range(len(dataframe)):
    img_name = dataframe.iloc[row, 0]
    img_data = np.array(dataframe.iloc[row, 1:])
    img_data = img_data.astype('float').reshape(137,236)
    img_data = cv2.resize(img_data, dsize=(img_dim, img_dim), interpolation=cv2.INTER_CUBIC)
    # normalizing the image
    img_data = (img_data*(255.0/img_data.max())).astype(np.uint8)
    # save
    plt.image.imsave('/home/ubuntu/datasets/bengali-ai/training_images/train_1/'+img_name+'.png', img_data, cmap='gray')

In [ ]:
del train_1

### Kaggle kernel 2 by numblebee

In [1]:
import numpy as np 
import os 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import cv2 
import pandas as pd 
from pathlib import Path

In [2]:
class TFRecordExtractor:
    def __init__(self, tfrecord_file, output_dir):
        self.tfrecord_file = os.path.abspath(tfrecord_file)
        self.count = 0
        self.output_dir = output_dir

    def _extract_fn(self,tfrecord):

        # Extract features using the keys set during creation
        feature = {'label': tf.FixedLenFeature([], tf.int64),
                'label_normal': tf.FixedLenFeature([], tf.int64),
                'image': tf.FixedLenFeature([], tf.string)}
        # Decode the record read by the reader
        features = tf.parse_single_example(tfrecord, features=feature)
        # Convert the image data from string back to the numbers
        image = tf.decode_raw(features['image'], tf.uint8)
        #image = tf.image.decode_png(features['image'],channels=1)
        label = features['label']
        label_normal = features['label_normal']
        image = tf.reshape(image, [299, 299, 1])
        return [image,label,label_normal]


    def post_process_images(self):
        image_data_list = self.get_images()
        b_c_df = pd.DataFrame(columns=['id', 'class', 'normal_class'])
        id_list = []
        class_list = []
        normal_class_list = []
        Path(self.output_dir).mkdir(parents=True, exist_ok=True)
        for image_data in image_data_list:
            self.count = self.count + 1
            # file_name = 'folder directory for writing images' + 'b_c' + str(
            #     self.count)
            file_name = os.path.join(self.output_dir, 'b_c' + str(self.count) )
            id_list.append('b_c' + str(
                self.count))
            class_list.append(image_data[1])
            normal_class_list.append(image_data[2])
            cv2.imwrite(file_name+".png",image_data[0])
        id_arr = np.array(id_list)
        class_arr = np.array(class_list)
        normal_class_arr = np.array(normal_class_list)
        b_c_df["id"]  = pd.Series(id_arr)
        b_c_df["class"] = pd.Series(class_arr)
        b_c_df["normal_class"] = pd.Series(normal_class_arr)
        b_c_df.to_csv(
            # path_or_buf='path to csv- example - training_3.csv',
            path_or_buf=os.path.join(self.output_dir, 'record.csv'),
            columns=['id', 'class', 'normal_class'])

    def get_images(self):
        # Initialize all tfrecord paths
        dataset = tf.data.TFRecordDataset([self.tfrecord_file])
        dataset = dataset.map(self._extract_fn)
        iterator = dataset.make_one_shot_iterator()
        next_element = iterator.get_next()
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            image_data_list = []
            try:
                while True:
                    image_data = sess.run(next_element)
                    image_data_list.append(image_data)
            except:
                pass

            return image_data_list

In [4]:
#t = TFRecordExtractor('~/datasets/mammography/training10_0/training10_0.tfrecords') 

In [5]:
dir_root = '~/datasets/mammography/training10_0/'
output_dir = os.path.join(dir_root, 'training_10_0') 
output_dir

'~/datasets/mammography/training10_0/training_10_0'

In [6]:
t = TFRecordExtractor(tfrecord_file= '~/datasets/mammography/training10_0/training10_0.tfrecords', output_dir=output_dir) 

In [7]:
t.post_process_images()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [18]:
import tensorflow as tf
import shutil
import numpy as np
import pandas as pd
import cv2

In [28]:
class TFRecordExtractor:
    def __init__(self, tfrecord_file):
        self.tfrecord_file = os.path.abspath(tfrecord_file)
        self.count = 0

    def _extract_fn(self,tfrecord):
        # Extract features using the keys set during creation
        feature = {'label': tf.FixedLenFeature([], tf.int64),
                'label_normal': tf.FixedLenFeature([], tf.int64),
                'image': tf.FixedLenFeature([], tf.string)}
        # Decode the record read by the reader
        features = tf.parse_single_example(tfrecord, features=feature)
        # Convert the image data from string back to the numbers
        image = tf.decode_raw(features['image'], tf.uint8)
        #image = tf.image.decode_png(features['image'],channels=1)
        label = features['label']
        label_normal = features['label_normal']
        image = tf.reshape(image, [299, 299, 1])
        return [image,label,label_normal]


    def post_process_images(self):
        image_data_list = self.get_images()
        b_c_df = pd.DataFrame(columns=['id', 'class', 'normal_class'])
        id_list = []
        class_list = []
        normal_class_list = []
        for image_data in image_data_list:
            self.count = self.count + 1
            file_name = '/home/ubuntu/datasets/mammography/training10_0/training_10_0/' + 'b_c' + str(
                self.count)
            id_list.append('b_c' + str(
                self.count))
            class_list.append(image_data[1])
            normal_class_list.append(image_data[2])
            cv2.imwrite(file_name+".png",image_data[0])
        id_arr = np.array(id_list)
        class_arr = np.array(class_list)
        normal_class_arr = np.array(normal_class_list)
        b_c_df["id"]  = pd.Series(id_arr)
        b_c_df["class"] = pd.Series(class_arr)
        b_c_df["normal_class"] = pd.Series(normal_class_arr)
        b_c_df.to_csv('/home/ubuntu/datasets/mammography/training10_0/training_10_0/training_3.csv')

    def get_images(self):
        # Initialize all tfrecord paths
        dataset = tf.data.TFRecordDataset([self.tfrecord_file])
        dataset = dataset.map(self._extract_fn)
        iterator = dataset.make_one_shot_iterator()
        next_element = iterator.get_next()
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            image_data_list = []
            try:
                while True:

                    image_data = sess.run(next_element)
                    image_data_list.append(image_data)
            except:
                pass

            return image_data_list

In [29]:
t = TFRecordExtractor('~/datasets/mammography/training10_0/training10_0.tfrecords')

In [30]:
t.post_process_images()